In [89]:
import pandas as pd
from pathlib import Path
import os
import openpyxl
from collections import defaultdict


In [90]:
repo_root = Path().cwd()
print(repo_root)
print(repo_root)

ref_path  = repo_root / "master_sheet_reference.xlsx"
data_path = repo_root / "assessment_guide.xlsx"


ref_sheets  = pd.read_excel(ref_path,  sheet_name=None)
data_sheets = pd.read_excel(data_path, sheet_name=None)

print("Reference file sheets:", list(ref_sheets.keys()))
print("Data file sheets     :", list(data_sheets.keys()))

c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\lighthouse_dash


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Valid

Reference file sheets: ['P&M Schedule', 'P&M RIsk', 'Cost Estimation & Optimization', 'D&C - Quality', 'D&C - Procurement', 'D&C - HSE & Welfare', 'Innovation & Technology', 'Design & Technical', 'D&C - Construction', 'Strategy & Operations', 'Visuals Menu']
Data file sheets     : ['Main', 'Instructions', 'Glossary&Definitions', 'DocumentList', 'DevCoAssessmentInput(P&M)', 'DevCoAssessmentAnalysis(P&M)', 'InterviewQuestion(P&M)', 'AddDataPoint(P&M)', 'DevCoAssessmentInput(D&T)', 'DevCoAssessmentAnalysis(D&T)', 'InterviewQuestion(D&T)', 'AddDataPoint(D&T)', 'DevCoAssessmentInput(D&C)', 'DevCoAssessmentAnalysis(D&C)', 'InterviewQuestion(D&C)', 'AddDataPoint(D&C)', 'DevCoAssessmentInput(CE&O)', 'DevCoAssessmentAnalysis(CE&O)', 'InterviewQuestion(CE&O)', 'AddDataPoint(CE&O)', 'DevCoAssessmentInput(I&T)', 'DevCoAssessmentAnalysis(I&T)', 'InterviewQuestion(I&T)', 'AddDataPoint(I&T)', 'DevCoAssessmentInput(S&O)', 'DevCoAssessmentAnalysis(S&O)', 'InterviewQuestion(S&O)', 'AddDataPoint(S&O)', '

c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [91]:
ref_sheets['P&M Schedule']

,Table,Column,Visual
0,AddDataPoint(P&M),Additional Data Point,Card
1,AddDataPoint(P&M),Value,Card
2,DevCoAssessmentAnalysis(P&M),Assessment Criteria,NaN
3,DevCoAssessmentAnalysis(P&M),Value,NaN
4,AddDataPoint(P&M),Year,NaN
5,AddDataPoint(P&M),Value Completed (Baseline) in M SAR,NaN
6,AddDataPoint(P&M),Value Completed (Actual) in M SAR,NaN
7,AddDataPoint(P&M),Value Completed (Forecast) in M SAR,NaN


In [92]:
data_sheets

{'Main':    Unnamed: 0  Unnamed: 1  Unnamed: 2  0.2  Unnamed: 4  Unnamed: 5  \
 0         NaN         NaN         NaN  0.8         NaN         NaN   
 
    Unnamed: 6  Unnamed: 7  Unnamed: 8  0.2.1  ...  0.2.4  Unnamed: 14  0.2.5  \
 0         NaN         NaN         NaN    NaN  ...    NaN          NaN    NaN   
 
    0.2.6  Unnamed: 17  MEINHARDT  Unnamed: 19  Unnamed: 20  Administrator:  \
 0    NaN          NaN        NaN          NaN          NaN             NaN   
 
    Admin  
 0    NaN  
 
 [1 rows x 23 columns],
 'Instructions':                 Unnamed: 0                     Unnamed: 1  \
 0   PIF Project Lighthouse                            NaN   
 1                      NaN                            NaN   
 2                      NaN                            NaN   
 3                       SN                           Page   
 4                        1                  Document List   
 5                        2        DevCo Assessment Input    
 6                      

In [93]:
ref_tabs = []     

for sheet_name, df in ref_sheets.items():
    if {"Table", "Column"}.issubset(df.columns):
        table_cols = (
            df.groupby("Table")["Column"]
              .apply(list)
              .to_dict()
        )
        ref_tabs.append({ sheet_name: table_cols })
    else:
        print(f"Skipping sheet {sheet_name!r}: missing \"Table\" and \"Columm\" columns")

ref_tabs

Skipping sheet 'Visuals Menu': missing "Table" and "Columm" columns


[{'P&M Schedule': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Year',
    'Value Completed (Baseline) in M SAR',
    'Value Completed (Actual) in M SAR',
    'Value Completed (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria', 'Value']}},
 {'P&M RIsk': {'AddDataPoint(P&M)': ['Additional Data Point',
    'Value',
    'Risk Category',
    'Number of Risk Items',
    'Risk',
    'Source',
    'Potential Impact',
    'Risk Rating',
    'Risk Status',
    'Current Measures'],
   'DevCoAssessmentAnalysis(P&M)': ['Assessment Criteria',
    'Value',
    'Rating']}},
 {'Cost Estimation & Optimization': {'AddDataPoint(CE&O)': ['Additional Data Point',
    'Value',
    'Year',
    'Budget Value (Baseline) in M SAR',
    'Budget Value (Revised) in M SAR',
    'Budget Value (Forecast) in M SAR'],
   'DevCoAssessmentAnalysis(CE&O)': ['Assessment Criteria', 'Value', 'Rating'],
   'DevCoAssessmentInput(CE&O)': ['Data Point', 'Input Value']}},
 {'D&C -

In [ ]:
pieces = []

for tab_dict in ref_tabs:
    sheet_key = next(iter(tab_dict.keys()))
    sheet_map = next(iter(tab_dict.values()))
    
    for table_name, cols in sheet_map.items(): 
        df = pd.read_excel(data_path, sheet_name=table_name, header=4) # pull in AG data
        df.columns = df.columns.str.strip()
        # print(cols) # ref columns
        # print(df.columns) # AG columns
        present = [c for c in cols if c in df.columns] # cols = columns from reference, present = columns in AG
        missing = set(cols) - set(present)
        if missing:
            print(f"⚠️ Tab {sheet_key}, in table {table_name!r}, these columns were missing: {missing}")
        
        df_sub = df[present].copy()
        for c in missing:
            df_sub[c] = pd.NA
    
        pieces.append(df_sub)

master_df = pd.concat(pieces, axis=1).reset_index(drop=True)

master_df.head(10)


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


⚠️ Tab D&C - Procurement, in table 'AddDataPoint(D&C)', these columns were missing: {nan}


c:\Users\jeremy.yap\OneDrive - Meinhardt Singapore Pte Ltd\Python\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Master shape: (78, 78)


,Additional Data Point,Value,Year,Value Completed (Baseline) in M SAR,Value Completed (Actual) in M SAR,Value Completed (Forecast) in M SAR,Assessment Criteria,Value,Additional Data Point,Value,...,Pending Approval,Year,Value_training,Additional Data Point,Value,Assessment Criteria,Value,Rating,Performance Signal Score,Performance Signal
0,Latest Approved Completion Date​,Q2-28,2025.0,0.0,NaN,NaN,DevCo Schedule Performance Index (SPI),0.9,Latest Approved Completion Date​,Q2-28,...,12.0,Jan,30.0,Implementation Success Rate for Innovation Str...,3.0,Is there an innovation Strategy or document th...,0.4,Needs Improvement,1.7,Innovation & Technology Deployment
1,Forecast Completion Date​,Q1-29,2026.0,1250.0,1250.0,NaN,NaN,NaN,Forecast Completion Date​,Q1-29,...,7.0,Feb,40.0,Innovation Adoption Effectiveness Index (strat...,1.0,NaN,NaN,NaN,NaN,NaN
2,Approved baseline completion date,Q3-27,2027.0,2000.0,1550.0,1550.0,% milestone achievement in accordance with the...,0.7,Approved baseline completion date,Q3-27,...,NaN,Mar,38.0,Innovation Adoption Effectiveness Index (skill...,3.0,NaN,NaN,NaN,NaN,NaN
3,No. of schedule iterations,8,2028.0,3100.0,NaN,2200.0,NaN,NaN,No. of schedule iterations,8,...,NaN,Apr,35.0,Innovation Adoption Effectiveness Index (plann...,2.0,NaN,NaN,NaN,NaN,NaN
4,Planned duration up to a specific phase as per...,120,2029.0,NaN,NaN,3100.0,% of projects with approved change requests in...,0.222222,Planned duration up to a specific phase as per...,120,...,NaN,May,0.0,Innovation Adoption Effectiveness Index (proce...,4.0,NaN,NaN,NaN,NaN,NaN
5,Duration taken to complete works up to the spe...,145,NaN,NaN,NaN,NaN,NaN,NaN,Duration taken to complete works up to the spe...,145,...,NaN,Jun,0.0,Innovation Adoption Effectiveness Index (partn...,2.0,NaN,NaN,NaN,NaN,NaN
6,Recovery Plan,Needs Improvement,NaN,NaN,NaN,NaN,% of projects with approved change requests in...,0.545455,Recovery Plan,Needs Improvement,...,NaN,Jul,0.0,Innovation Adoption Effectiveness Index (resou...,0.6,NaN,NaN,NaN,NaN,NaN
7,Extent of all EOT (design stage),3,NaN,NaN,NaN,NaN,NaN,NaN,Extent of all EOT (design stage),3,...,NaN,Aug,0.0,Implementation Success Rate for Smart City Str...,3.0,NaN,NaN,NaN,NaN,NaN
8,Extent of all EOT (const. stage),6,NaN,NaN,NaN,NaN,% of projects with approved schedule extension,0.65,Extent of all EOT (const. stage),6,...,NaN,Sep,0.0,Number of Smart City Use Cases,10.0,Is there a smart city Strategy or document tha...,0.4,Needs Improvement,NaN,NaN
9,No. of unresolved issues,7,NaN,NaN,NaN,NaN,NaN,NaN,No. of unresolved issues,7,...,NaN,Oct,0.0,Smart City Adoption Effectiveness Index (strat...,2.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
output_path = repo_root/ 'master_sheet.xlsx'
master_df.to_excel(output_path, index=False)

print(f"✅ Saved master sheet to {output_path!r}")

✅ Saved master sheet to WindowsPath('c:/Users/jeremy.yap/OneDrive - Meinhardt Singapore Pte Ltd/Python/lighthouse_dash/master_sheet.xlsx')
